![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
########## IMPORTATIONS BLOCK ##########

import time
import pandas as pd
import plotly.graph_objects as go

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
########## FILE OPENING BLOCK (USING GOOGLE COLAB) ##########

filename_features = "features.txt"
filename_labels = "activity_labels.txt"
filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"
filename_subtest = "subject_test.txt"
filename_xtest = "X_test.txt"
filename_ytest = "y_test.txt"

########## FILE READING BLOCK ##########

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze("columns")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")

X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [3]:
# Model configuration
tree = DecisionTreeClassifier(ccp_alpha=0.001)

# Training & timing
%time tree.fit(X_train, y_train)

# Prediction & evaluation
train_pred = tree.predict(X_train)
test_pred = tree.predict(X_test)
train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Training accuracy: {train_accuracy * 100: .1f}%")
print(f"Testing accuracy: {test_accuracy * 100: .1f}%")

CPU times: user 6.96 s, sys: 19.9 ms, total: 6.98 s
Wall time: 10.3 s
Training accuracy:  97.6%
Testing accuracy:  88.1%


## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [4]:
# Performing PCA with only one component
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Timming the training
%time tree.fit(X_train_pca, y_train)

# Prediction & evaluation
train_pred_pca = tree.predict(X_train_pca)
test_pred_pca = tree.predict(X_test_pca)
train_accuracy_pca = accuracy_score(y_train, train_pred_pca)
test_accuracy_pca = accuracy_score(y_test, test_pred_pca)

print(f"Training accuracy PCA (1 component): {train_accuracy_pca * 100: .1f}%")
print(f"Testing accuracy (1 component): {test_accuracy_pca * 100: .1f}%")

CPU times: user 119 ms, sys: 15.3 ms, total: 134 ms
Wall time: 252 ms
Training accuracy PCA (1 component):  50.0%
Testing accuracy (1 component):  45.7%


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [7]:
# Inicializing the list to insert the informations
results = []

# Same code but running throught all numbers of components needed
for n_components in [1, 2, 5, 10, 50]:
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    tree = DecisionTreeClassifier(ccp_alpha=0.001)

    start_time = time.time()
    tree.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    train_pred = tree.predict(X_train_pca)
    test_pred = tree.predict(X_test_pca)
    train_accuracy = accuracy_score(y_train, train_pred)
    test_accuracy = accuracy_score(y_test, test_pred)

    # Storing each result on the df
    results.append({'Components': n_components,
                    'Train Accuracy (%)': train_accuracy * 100,
                    'Test Accuracy (%)': test_accuracy * 100,
                    'Processing Time (s)': training_time})

# Using Pandas to better visualize the results and able its easily comparison and engineering
results_df = pd.DataFrame(results)
results_df.set_index('Components', inplace=True)
results_df.style.format(precision=2, decimal=',')

,Train Accuracy (%),Test Accuracy (%),Processing Time (s)
Components,,,
1,"49,97","45,71","0,30"
2,"61,28","58,47","0,16"
5,"84,60","78,86","0,09"
10,"89,32","82,29","0,19"
50,"91,95","81,54","0,89"


In [21]:
# Plotting the curves along the components
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Train Accuracy (%)'], mode='lines+markers', name='Train', marker=dict(color='gray')))
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Test Accuracy (%)'], mode='lines+markers', name='Test', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Processing Time (s)'], mode='lines+markers', name='Processing Time', marker=dict(color='red'), yaxis='y2'))

# Setting adjustments
fig.update_layout(title='Accuracies and Processing Time by Components',
                  font=dict(size=24,
                            color='black',
                            family='Calibri'),
                  xaxis_title='Number of Components (-)',
                  yaxis_title='Accuracy (%)',
                  yaxis2=dict(title='Time (s)',
                              overlaying='y',
                              side='right'),
                  legend=dict(x=0.375, y=1.1, orientation='h'),
                  template='simple_white',
                  title_x=0.5)
fig.show()

## O que aconteceu com a acurácia?

> Atingiu no máximo, cerca de 82% de ajuste ao conjunto teste, sendo que essa eficiência foi obtida com cerca de apenas 10 componentes.

## O que aconteceu com o tempo de processamento?

> Parece que a taxa de crescimento está aumentando gradualmente, indicando um crescimento não linear. Essa característica é típica de um crescimento exponencial ou polinomial (onde a taxa de mudança não é constante)

## Conclusão:

> Naturalmente, ao inserirmos mais componentes, o custo de tempo aumentaria não linearmente, mas a acurácia da árvore se manteria próxima dos 80% no conjunto teste, resultando em um trade-off negativo.
>
> Podemos dizer para essa análise de PCA, n=5 é a quantidade ideal de componentes para esse conjunto de dados usando ccp_alpha=0,001, pois possui o menor tempo de processamento frente a acurácia obtida.